In [20]:
from IPython.core.display import HTML ; HTML(open("../custom.css", "r").read()) # notebook style

# Convergence to uniform flow in a duct

This notebook uses `pyfvm` python package and demonstrate the convergence to a uniform flow in a duct, given inlet and outlet boundary conditions.

In [21]:
import time
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
from pyfvm.mesh  import *
from pyfvm.field import *
from pyfvm.xnum  import *
from pyfvm.integration import *
import pyfvm.modelphy.euler as euler
import pyfvm.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 100
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

In [22]:
nx = 100
meshsim  = unimesh(ncell=nx,  length=10.)

model = euler.model()
bcL = { 'type': 'insub',  'ptot': 1.4, 'rttot': 1. }
bcR = { 'type': 'outsub', 'p': 1. }

rhs = modeldisc.fvm(model, meshsim, muscl(vanalbada), 
      bcL=bcL, bcR=bcR)
solver = rk3ssp(meshsim, rhs)

# computation
#
nsol    = 100
endtime = 200.
cfl     = .5

finit = rhs.fdata_fromprim([  1., 0., 1. ]) # rho, u, p

fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))

solver.show_perf()
mach_th = np.sqrt(((bcL['ptot']/bcR['p'])**(1./3.5)-1.)/.2)
print("theoretical Mach: {}".format(mach_th))

In [23]:
# Figure / Plot of final results
varname='mach'
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.set_ylabel(varname) ; ax1.set_ylim(0., 1.1*np.max(fsol[-1].phydata(varname)))
ax1.grid(linestyle='--', color='0.5')
line1, = fsol[-1].plot(varname, 'k-', axes=ax1)
ax2.set_ylabel('error') ; ax2.set_ylim(1e-5, 1.)
ax2.grid(linestyle='--', color='0.5')
ttime = [ fsol[i].time for i in range(nsol+1) ]
error = [ np.sqrt(np.sum((fsol[i].phydata(varname)-mach_th)**2)/nx)/mach_th for i in range(nsol+1) ]
#print error
line2, = ax2.semilogy(ttime, error)


In [24]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    fsol[k].set_plotdata(line1, varname)
    line2.set_data(ttime[0:k], error[0:k])
    return line1, line2

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100, blit=True)
ani
#from IPython.display import HTML
#HTML(ani.to_html5_video()) # if no rcparams